In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import scipy.optimize as opt

from AppliedMacroPlots import *

In [ ]:
# Parameter values

α = 0.33
δ = 0.1
n = 0.01
g = 0
σ = 2
β = 0.96

c_min = 0.49
# c_min = 0

In [ ]:
# Define needed functions

def ProductionFunction(k):
    return k**α

def InterestRate(k):
    return α*k**(α-1) - δ
    
def SteadyState():
    
    r_star = (1+n) * (1+g)**σ / β - 1
    k_star = (α/(r_star+δ))**(1/(1-α))
    c_star = ProductionFunction(k_star) - (δ+n+g+n*g)*k_star
    
    return k_star, c_star

def ResourceConstraint(k, c):
    
    return (ProductionFunction(k) + (1-δ)*k - c)/((1+n)*(1+g))

def EulerEquation(k, c):
    
    k_next = ResourceConstraint(k, c)
    
    if k_next > 0:
        r = InterestRate(k_next)
        c_next = (β*(1+r)/(1+n))**(1/σ) * (c-c_min) / (1+g) + c_min
        return c_next
    else:
        return 0

In [ ]:
# Forward equations and convergence criterion

def Path(c_0, k_0, T=100):
    
    T += 1
    
    k_t = np.zeros(T)
    c_t = np.zeros(T)
    
    k_t[0] = k_0
    c_t[0] = c_0
    
    for t in range(T-1):
        k_t[t+1] = ResourceConstraint(k_t[t], c_t[t])
        if k_t[t+1] > 0:
            c_t[t+1] = EulerEquation(k_t[t], c_t[t])
        else:
            k_t[t+1] = 0
            c_t[t+1] = 0
            
    return k_t, c_t

def Path_crit(c_0, k_0, T=100):
    
    k_t, c_t = Path(c_0, k_0, T)
    k_star, c_star = SteadyState()
    
    ss_diff = np.sqrt((k_t-k_star)**2 + (c_t-c_star)**2)
    
    return np.min(ss_diff) + ss_diff[-1]

In [ ]:
# Steady state
print(SteadyState())
k_star, c_star = SteadyState()

In [ ]:
k_0 = k_star / 20

In [ ]:
# Find the function minimum, starting from an initial guess
result = opt.minimize(Path_crit, 0.5, args=(k_0, 100), method='Nelder-Mead')
print(result)

c_0 = result.x

In [ ]:
k_t, c_t = Path(c_0, k_0, 100)

y_t = ProductionFunction(k_t)

s_t = 1 - c_t/y_t

g_y_t = 100*(y_t[1:]/y_t[:-1]-1)

In [ ]:
plt.plot(k_t, lw=2)
plt.show()

plt.plot(c_t, lw=2)
plt.show()

plt.plot(s_t, lw=2)
plt.show()

plt.plot(g_y_t, lw=2)
plt.show()

In [ ]:
plt.scatter(np.log(y_t[:-1]), g_y_t)
plt.show()